In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from xgboost.spark import SparkXGBClassifier
# from xgboost import XGBoostClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col
from sklearn.metrics import f1_score, recall_score

In [2]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark
spark_object = init_spark()

23/03/15 18:40:00 WARN Utils: Your hostname, karyubuntu resolves to a loopback address: 127.0.1.1; using 172.31.170.249 instead (on interface wlp0s20f3)
23/03/15 18:40:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 18:40:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_spark = spark_object.read.option("delimiter", ",").csv("./data/trainbalanced.csv", header='true', inferSchema='true')

In [4]:
for i in train_spark.columns:
    oldname = i
    if "." in oldname:
        newname = oldname.replace(".","")
        train_spark = train_spark.withColumnRenamed(oldname, newname)

In [5]:
train_data, test_data = train_spark.randomSplit([0.8, 0.2], seed=123)

In [6]:
input_list =[]
for i in train_spark.columns:
    if i != 'isFraud':
        input_list.append(i)

In [7]:
assembler = VectorAssembler(inputCols=input_list,outputCol='features')

In [8]:
train_features = assembler.transform(train_data)
test_features = assembler.transform(test_data)

In [12]:
xgb = SparkXGBClassifier(
    features_col="features",
    label_col="isFraud",
    num_round=10,
    max_depth=3,
    eta=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    seed=42
)

In [10]:
# xgb = SparkXGBClassifier(label_col="cnt",max_depth=5, missing=0.0,
#     validation_indicator_col='isVal', weight_col='weight',
#     early_stopping_rounds=1, eval_metric='logloss')

# new_xgb = SparkXGBClassifier(max_depth=5, missing=0.0,
# validation_indicator_col='isVal', weight_col='weight',
# early_stopping_rounds=1, eval_metric='logloss')

In [13]:
try:
    model = xgb.fit(train_features)
except Exception as e:
    print(e)

[18:44:35] task 0 got new rank 0                                    (0 + 1) / 1]
[18:44:39] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


In [14]:
predictions = model.transform(test_features)

In [15]:
y_pred=predictions.select("prediction").collect()
y_orig=predictions.select("isFraud").collect()

In [16]:
print(f1_score(y_orig, y_pred))
print(recall_score(y_orig, y_pred))

0.9677555321390938
0.9516201959306707


In [ ]:
# evaluator = BinaryClassificationEvaluator(labelCol="isFraud")
# accuracy = evaluator.evaluate(predictions)

# print(f"Accuracy: {accuracy:.4f}")